# core

> implements zero mean GARCH(1,1) model with t noise distribution

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
from functools import cached_property
from typing import Literal
import numpy as np
from numba import njit
from arch import arch_model

In [ ]:
#|export

@njit
def _get_vs(y, params, sig2_init=1.0):
    """
    not clear how arch package initialize the conditional variance at time 0
    here kick off the computation with value 1.0
    it is provably true that vs computed below
    is insensitive to the initial value (coupling exponentially fast)
    """
    om,al,be = params
    vs = np.full(y.size, sig2_init)
    for i in range(1,y.size):
        vs[i] = om + al * y[i-1]**2 + be * vs[i-1]
    return vs

In [ ]:
#|export

@njit
def _make_fcst_vs(params,last_v, last_y, horizon):
    om,al,be = params
    fcst_vs = np.empty(horizon)
    fcst_vs[0] = om + al* last_y**2 + be* last_v
    for i in range(1,horizon):
        fcst_vs[i] = om + (al+be) * fcst_vs[i-1]
    return fcst_vs


In [ ]:
#|export

@njit(fastmath=True)
def _simulate(params, last_y, last_v, horizon, n_rep, seed, ws=None):
    om,al,be = params
    if ws is None:
        np.random.seed(seed)
        ws = np.random.randn(n_rep,horizon)
    y = np.empty((n_rep, horizon+1))
    v = np.empty((n_rep, horizon+1))
    y[:,0] = last_y
    v[:,0] = last_v
    for i in range(1,horizon+1):
        v[:,i] = om + al* y[:,i-1]**2 + be * v[:,i-1]
        y[:,i] = np.sqrt(v[:,i]) * ws[:,i-1]
        # used fastmath for sqrt; but not sure if this makes a difference
    return y[:,1:]


In [ ]:
#|export

class GARCHETTE:
    """simple garch model"""
    def __init__(self):
        self._y = None
        self.params = None
        self.dist_params = None
        self._is_fit = False
        self._v_init = 1.0

    def fit(self,y:np.ndarray):
        """fit y to garch"""
        self._y = y
        res = arch_model(y, mean="Zero", dist="t", rescale=False).fit(disp="off")
        om,al,be,nu = res.params
        self.params = np.array((om,al,be))
        self.dist_params = np.array(nu)
        self._v_init = res.conditional_volatility[0]**2  ## feed the initial value of arch_model impl to be coherent
        self._is_fit = True
        return self

    @property
    def vs(self) -> np.ndarray:
        """property: estimated conditional variance, same shape as y"""
        assert self._is_fit
        return _get_vs(self._y, self.params, sig2_init = self._v_init)

    @property
    def std_resids(self) -> np.ndarray:
        """property: estimated standardized residual"""
        assert self._is_fit
        return self._y / np.sqrt(self.vs)

    def forecast_vs(self, 
                    horizon:int
                    ) -> np.ndarray:
        """forecast conditional variance in the horizon"""
        assert self._is_fit
        return _make_fcst_vs(self.params, self.vs[-1], self._y[-1], horizon)

    def simulate(self, 
                 horizon:int, # path length
                 method:Literal["bootstrap","simulate"]="simulate",# "bootstrap" resamples from past std_resids; "simulate" simulates gaussian nosie
                 n_rep=1_000, # number of repetitions
                 seed=42) -> np.ndarray: 
        assert self._is_fit
        if method == "bootstrap":
            np.random.seed(seed)
            ws = np.random.choice(self.std_resids, size=(n_rep, horizon),replace=True)
        else: ws=None
        return _simulate(self.params,self._y[-1], self.vs[-1], horizon, n_rep, seed, ws)

In [ ]:
show_doc(GARCHETTE.fit)

---

[source](https://github.com/xiaochuany/archette/blob/main/archette/core.py#L66){target="_blank" style="float:right; font-size:smaller"}

### GARCHETTE.fit

>      GARCHETTE.fit (y:numpy.ndarray)

*fit y to garch*

In [ ]:
show_doc(GARCHETTE.vs)

---

[source](https://github.com/xiaochuany/archette/blob/main/archette/core.py#L76){target="_blank" style="float:right; font-size:smaller"}

### GARCHETTE.vs

>      GARCHETTE.vs ()

*property: estimated conditional variance, same shape as y*

In [ ]:
show_doc(GARCHETTE.std_resids)

---

[source](https://github.com/xiaochuany/archette/blob/main/archette/core.py#L81){target="_blank" style="float:right; font-size:smaller"}

### GARCHETTE.std_resids

>      GARCHETTE.std_resids ()

*property: estimated standardized residual*

In [ ]:
show_doc(GARCHETTE.forecast_vs)


---

[source](https://github.com/xiaochuany/archette/blob/main/archette/core.py#L85){target="_blank" style="float:right; font-size:smaller"}

### GARCHETTE.forecast_vs

>      GARCHETTE.forecast_vs (horizon:int)

*forecast conditional variance in the horizon*

In [ ]:
show_doc(GARCHETTE.simulate)

---

[source](https://github.com/xiaochuany/archette/blob/main/archette/core.py#L89){target="_blank" style="float:right; font-size:smaller"}

### GARCHETTE.simulate

>      GARCHETTE.simulate (horizon:int,
>                          method:Literal['bootstrap','simulate']='simulate',
>                          n_rep=1000, seed=42)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| horizon | int |  | path length |
| method | Literal | simulate | "bootstrap" resamples from past std_resids; "simulate" simulates gaussian nosie |
| n_rep | int | 1000 | number of repetitions |
| seed | int | 42 |  |
| **Returns** | **ndarray** |  |  |

In [ ]:
y = np.random.randn(300)
GARCHETTE().fit(y).simulate(20, n_rep=100).shape

(100, 20)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()